In [ ]:
from keras.datasets import mnist
import matplotlib.pyplot as plt
import numpy as np

(train_X, train_y), (test_X, test_y) = mnist.load_data()
def zero_pad(X, pad):
  return np.pad(X, ((0,0), (pad,pad),(pad,pad)))
#Notes
# Add activation function??? Will need to consider this in the backward pass
train_X = zero_pad(train_X, 2)
test_X = zero_pad((test_X), 2)


11490434/11490434 [==============================] - 0s 0us/step


In [ ]:
train_X = train_X / 255
test_X = test_X / 255

test_X = test_X
test_y = test_y

In [ ]:
W1 = np.random.randn(5,5,6)

b1 = np.random.randn(1,1,6)

hparameters1 = {'pad': 0,
               "stride": 1}
hparameters2 = {'stride':2, 'f':2}
W2 = np.random.randn(5,5,6,16)
b2 = np.random.rand(1,1,1,16)
hparameters3 = {'pad': 0, 'stride': 1}
hparameters4 = {'stride': 2, 'f':2}
W3 = np.random.randn(5,5,16,120)
b3 = np.random.randn(1,1,1,120)
hparameters5 = {'pad': 0, 'stride': 1}
W4 = np.random.randn(84,120)
b4 = np.random.randn(84 , 1)
W5 = np.random.randn(10, 84)
b5 = np.random.randn(10, 1)

In [ ]:
import copy

10000

In [ ]:
for i in range(10):
# First layer - Conv
  A_prev = train_X
  Y = train_y
  Z1, cache_conv1 = conv_forward2D(A_prev, W1, b1, hparameters1)


  # Second layer - Pool
  P1, cache_pool1 = pool_forward(Z1, hparameters2, mode="max")

  # Third layer - Conv
  Z2, cache_conv2 = conv_forward3D(P1, W2, b2, hparameters3)

  # Fourth layer - Pool
  P2, cache_pool2 = pool_forward(Z2, hparameters4, mode='max')


  # Fifth layer - Conv
  Z3, cache_conv3 = conv_forward3D(P2, W3, b3, hparameters5)


  # Flatten and Fully Connected Layers
  X = np.squeeze(Z3).T


  Z4 = W4.dot(X) + b4


  A1 = tanh(Z4)

  Z5 = W5.dot(A1) + b5


  A2 = softmax(Z5)


  #Fully connceted backprop
  dW4, db4, dW5, db5, dZ4 = backward_prop(Z4, A1, Z5, A2, W4, W5, X, Y)

  #Conv/pool backprops
  dZ3, dW3, db3 = conv_backward(dZ4.T,cache_conv3)
  dP2 = pool_backward(dZ3,cache_pool2)

  dZ2, dW2, db2 = conv_backward3D(dP2,cache_conv2)

  dP1 = pool_backward(dZ2,cache_pool1)

  dZ1, dW1,db1 = conv_backward2D(dP1,cache_conv1)

  lr = 0.1
  W1 = W1 - lr * dW1
  b1 = b1 - lr * db1
  W2 = W2 - lr * dW2
  b2 = b2 - lr * db2
  W3 = W3 - lr * dW3
  b3 = b3 - lr * db3
  W4 = W4 - lr * dW4
  b4 = b4 - lr * db4
  W5 = W5 - lr * dW5
  b5 = b5 - lr * db5



  # Step 1: Obtain Predictions
  predictions = forward_pass(test_X, W1, b1, W2, b2, W3, b3, W4, b4, W5, b5)

  # Step 2: Convert predictions to class labels
  predicted_labels = np.argmax(predictions, 0)
  print(predicted_labels)
  # Step 3: Compute Accuracy
  Y1 = Y[:10000]
  accuracy = np.sum(predictions == Y1) / Y1.size

  print("Accuracy:", accuracy)


<ipython-input-3-e64eb903140f>:82: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  Z[i, h, w, c] = tanh(conv_single_step(a_slice_prev, weights, biases))
<ipython-input-3-e64eb903140f>:138: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  Z[i, h, w, c] = tanh(conv_single_step(a_slice_prev, weights, biases))


[2 2 2 ... 8 2 3]
Accuracy: 0.0


In [ ]:
# Forward pass through the network
def one_hot(Y):
    x = int(Y.size)
    y = int(Y.max())
    one_hot_Y = np.zeros((x, y + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return np.asarray(one_hot_Y)
def forward_pass(X, W1, b1, W2, b2, W3, b3, W4, b4, W5, b5):
    # First layer - Conv
    Z1, cache_conv1 = conv_forward2D(X, W1, b1, hparameters1)
    # Second layer - Pool
    P1, cache_pool1 = pool_forward(Z1, hparameters2, mode="max")
    # Third layer - Conv
    Z2, cache_conv2 = conv_forward3D(P1, W2, b2, hparameters3)
    # Fourth layer - Pool
    P2, cache_pool2 = pool_forward(Z2, hparameters4, mode='max')
    # Fifth layer - Conv
    Z3, cache_conv3 = conv_forward3D(P2, W3, b3, hparameters5)
    # Flatten the output for fully connected layers
    X_flatten = np.squeeze(Z3).T
    # Sixth layer - FCL
    Z4 = W4.dot(X_flatten) + b4
    A1 = tanh(Z4)
    # Seventh layer
    Z5 = W5.dot(A1) + b5
    A2 = softmax(Z5)
    return A2


def backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y):
    m = Y.size
    one_hot_Y = one_hot(Y)
    dZ2 = A2 - one_hot_Y
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2)
    dZ1 = W2.T.dot(dZ2) * deriv_tanh(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1)
    return dW1, db1, dW2, db2, dZ1


def tanh(Z):
  return np.tanh(Z)

def deriv_tanh(Z):
  return 1 - tanh(Z)*tanh(Z)

def softmax(Z):
    e_Z = np.exp(Z - np.max(Z))  # Normalize inputs to prevent overflow
    return e_Z / np.sum(e_Z)


def conv_single_step(a_slice_prev, W, b):
  s = np.multiply(a_slice_prev, W)
  Z = np.sum(s)
  Z = Z + b
  return Z

def conv_forward2D(A_prev, W, b, hparameters):
  #(m, n_H_prev, n_W_prev, n_C_prev) = A_prev.shape
  (m, n_H_prev, n_W_prev) = A_prev.shape
  #(f, f,n_C_prev,  n_C) = W.shape
  (f, f,n_C) = W.shape
  stride = hparameters['stride']
  pad = hparameters['pad']
  n_H = int((n_H_prev - f + 2*pad)/stride + 1)
  n_W = int((n_W_prev - f + 2 * pad)/stride + 1)
  # 512 - 7/2 + 1
  Z = np.zeros((m, n_H, n_W, n_C))
  for i in range(m):
    a_prev_pad = A_prev[i]
    for h in range(n_H):
      vert_start = h *stride
      vert_end = vert_start + f
      for w in range(n_W):
        horiz_start = w * stride
        horiz_end = horiz_start + f
        for c in range(n_C):
          a_slice_prev = a_prev_pad[vert_start:vert_end, horiz_start:horiz_end]
          weights = W[:,:,c]
          biases = b[:,:,c]
          Z[i, h, w, c] = tanh(conv_single_step(a_slice_prev, weights, biases))
  cache = (A_prev, W, b, hparameters)

  return Z, cache


def pool_forward(A_prev, hparameters, mode = "max"):
  (m, n_H_prev, n_W_prev, n_C_prev) = A_prev.shape
  f = hparameters['f']
  stride = hparameters['stride']
  n_H = int(1 + (n_H_prev - f) / stride)
  n_W = int(1 + (n_W_prev - f) / stride)
  n_C = n_C_prev
  A = np.zeros((m, n_H, n_W, n_C))
  for i in range(m):
    a_prev_slice = A_prev[i]
    for h in range(n_H):
      vert_start = stride * h
      vert_end = vert_start + f
      for w in range(n_W):
        horiz_start = stride * w
        horiz_end = horiz_start + f
        for c in range(n_C):
          a_slice_prev = a_prev_slice[vert_start:vert_end, horiz_start:horiz_end, c]
          if mode == 'max':
            A[i,h,w,c] = np.max(a_slice_prev)
          elif mode == 'mean':
            A[i,h,w,c] = np.mean(a_slice_prev)

  cache = (A_prev, hparameters)

  # Making sure your output shape is correct
  assert(A.shape == (m, n_H, n_W, n_C))

  return A, cache

def conv_forward3D(A_prev, W, b, hparameters):
  (m, n_H_prev, n_W_prev, n_C_prev) = A_prev.shape
  (f, f,n_C_prev,  n_C) = W.shape
  stride = hparameters['stride']
  pad = hparameters['pad']
  n_H = int((n_H_prev - f + 2*pad)/stride + 1)
  n_W = int((n_W_prev - f + 2 * pad)/stride + 1)
  Z = np.zeros((m, n_H, n_W, n_C))
  for i in range(m):
    a_prev_pad = A_prev[i]
    for h in range(n_H):
      vert_start = h *stride
      vert_end = vert_start + f
      for w in range(n_W):
        horiz_start = w * stride
        horiz_end = horiz_start + f
        for c in range(n_C):
          a_slice_prev = a_prev_pad[vert_start:vert_end, horiz_start:horiz_end]
          weights = W[:,:,:,c]
          biases = b[:,:,:,c]
          Z[i, h, w, c] = tanh(conv_single_step(a_slice_prev, weights, biases))

  cache = (A_prev, W, b, hparameters)

  return Z, cache




def conv_backward3D(dZ, cache):

  A_prev, W, b, hparameters = cache

  m, n_H_prev, n_W_prev, n_C_prev = A_prev.shape

  f, f, n_C_prev, n_C = W.shape

  stride = hparameters['stride']
  pad = hparameters['pad']

  m, n_H, n_W, n_C = dZ.shape

  dA_prev = np.zeros((m, n_H_prev, n_W_prev, n_C_prev))
  dW = np.zeros((f,f, n_C_prev, n_C))
  db = np.zeros((1,1,1,n_C))

  A_prev_pad = A_prev
  dA_prev_pad = dA_prev

  for i in range(m):
      a_prev_pad = A_prev_pad[i]
      da_prev_pad = dA_prev_pad[i]

      for h in range(n_H):
        for w in range(n_W):
          for c in range(n_C):

            vert_start = stride * h
            vert_end = vert_start + f
            horiz_start = stride * w
            horiz_end = horiz_start + f
            a_slice = a_prev_pad[vert_start:vert_end, horiz_start:horiz_end]
            da_prev_pad[vert_start:vert_end, horiz_start:horiz_end, :] += (W[:,:,:,c] * dZ[i, h, w, c]) * deriv_tanh(a_slice)
            dW[:,:,:,c] += a_slice * dZ[i, h, w, c]
            db[:,:,:,c] += dZ[i, h, w, c]

      dA_prev[i, :, :, :] = da_prev_pad

  assert(dA_prev.shape == (m, n_H_prev, n_W_prev, n_C_prev))

  return dA_prev, dW, db

In [ ]:
def conv_backward2D(dZ, cache):
  A_prev, W, b, hparameters = cache
  m, n_H_prev, n_W_prev = A_prev.shape
  f, f, n_C = W.shape
  stride = hparameters['stride']
  pad = hparameters['pad']
  m, n_H, n_W, n_C = dZ.shape
  dA_prev = np.zeros((m, n_H_prev, n_W_prev))
  dW = np.zeros((f, f, n_C))
  db = np.zeros((1,1,n_C))
  A_prev_pad = zero_pad(A_prev, pad)
  dA_prev_pad = zero_pad(dA_prev,pad)

  for i in range(m):

    a_prev_pad = A_prev_pad[i]
    da_prev_pad = dA_prev_pad[i]
    for h in range(n_H):
      for w in range(n_W):
        for c in range(n_C):
          vert_start = stride * h
          vert_end = vert_start + f
          horiz_start = stride * w
          horiz_end = horiz_start + f
          a_slice = a_prev_pad[vert_start:vert_end, horiz_start:horiz_end]

          #I might need to get
          da_prev_pad[vert_start:vert_end, horiz_start:horiz_end] += (W[:,:,c] * dZ[i, h, w, c]) * deriv_tanh(a_slice)
          dW[:,:,c] += a_slice * dZ[i, h, w, c]
          db[:,:,c] += dZ[i, h, w, c]

    dA_prev[i, :, :] = da_prev_pad


  assert(dA_prev.shape == (m, n_H_prev, n_W_prev))

  return dA_prev, dW, db



def conv_backward(dZ, cache):

  A_prev, W, b, hparameters = cache

  m, n_H_prev, n_W_prev, n_C_prev = A_prev.shape

  f, f, n_C_prev, n_C = W.shape

  stride = hparameters['stride']
  pad = hparameters['pad']

  m, n_C = dZ.shape
  n_H = 1
  n_W = 1

  dA_prev = np.zeros((m, n_H_prev, n_W_prev, n_C_prev))
  dW = np.zeros((f,f, n_C_prev, 120))
  db = np.zeros((1,1,1,120))

  A_prev_pad = A_prev
  dA_prev_pad = dA_prev

  for i in range(m):
      a_prev_pad = A_prev_pad[i]
      da_prev_pad = dA_prev_pad[i]

      for h in range(n_H):
        for w in range(n_W):
          for c in range(n_C):

            vert_start = stride * h
            vert_end = vert_start + f
            horiz_start = stride * w
            horiz_end = horiz_start + f
            a_slice = a_prev_pad[vert_start:vert_end, horiz_start:horiz_end]
            da_prev_pad[vert_start:vert_end, horiz_start:horiz_end, :] += W[:,:,:,c] * dZ[i, c]
            dW[:,:,:,c] += a_slice * dZ[i, c]
            db[:,:,:,c] += dZ[i, c]

      dA_prev[i, :, :, :] = da_prev_pad

  assert(dA_prev.shape == (m, n_H_prev, n_W_prev, n_C_prev))

  return dA_prev, dW, db
def create_mask_from_window(x):
    mask = (x == np.max(x))
    return mask


def backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y):
    m = Y.size
    one_hot_Y = one_hot(Y)
    dZ2 = A2 - one_hot_Y
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2)
    dZ1 = W2.T.dot(dZ2) * deriv_tanh(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1)
    return dW1, db1, dW2, db2, dZ1



def pool_backward(dA, cache, mode = "max"):
    (A_prev, hparameters) = cache

    stride = hparameters['stride']
    f = hparameters['f']
    m, n_H_prev, n_W_prev, n_C_prev = A_prev.shape
    m, n_H, n_W, n_C = dA.shape
    dA_prev = np.zeros((m, n_H_prev, n_W_prev, n_C_prev))
    for i in range(m): # loop over the training examples
        a_prev = A_prev[i]
        for h in range(n_H):                   # loop on the vertical axis
            for w in range(n_W):               # loop on the horizontal axis
                for c in range(n_C):           # loop over the channels (depth)
                    vert_start = stride * h
                    vert_end = vert_start + f
                    horiz_start = stride * w
                    horiz_end = horiz_start + f
                    if mode == "max":
                        a_prev_slice = a_prev[vert_start:vert_end, horiz_start:horiz_end, c]
                        mask = create_mask_from_window(a_prev_slice)
                        dA_prev[i, vert_start: vert_end, horiz_start: horiz_end, c] += mask * dA[i,h,w,c]
                    elif mode == "average":
                        da = dA[i,h,w,c]
                        shape = (f,f)
                        dA_prev[i, vert_start: vert_end, horiz_start: horiz_end, c] +=  distribute_value(da, shape)
    assert(dA_prev.shape == A_prev.shape)
    return dA_prev


def distribute_value(dz, shape):

    (n_H, n_W) = shape
    average = dz / (n_H * n_W)
    a = np.ones(shape) * average

    return a